<a href="https://colab.research.google.com/github/ali1903/Android-Programming/blob/master/ANN_keras_Train_Test_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Google Drive Baglantisi**


In [ ]:
#Google Drive baglanti
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/Colab Notebooks/Artificial Neural Network/
%ls

/content/gdrive/My Drive/Colab Notebooks/Artificial Neural Network
93.22.h5                                           ANN_keras-Train-Test.ipynb
93.25.h5                                           ANN_scikit.ipynb
98.15.h5                                           deneme.h5
ANN_keras.ipynb                                    diabetes.csv
ANN_keras-Train-Test-Final-FeatureSelection.ipynb  Final_Veriseti.csv
ANN_keras-Train-Test-Final.ipynb                   Numara_AdSoyad.docx


**Verileri Oluştur**

In [ ]:
#Gereken kutuphaneler yuklenmeli
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split  #Rastgele Train Test Bolmek icin
from sklearn.preprocessing import MinMaxScaler        #Min-Max normalization yapmak icin
import numpy

numpy.random.seed(7)
#veriseti yukleme
dataset=numpy.loadtxt("Final_Veriseti.csv", delimiter=",")

#datalar 0:16 arasi featurelar alindi
datalar=dataset[:,0:16]
datalar=MinMaxScaler().fit_transform(datalar)
#Veriler [0 1] araligina normalize edildi

etiketler=dataset[:,16]
#etiketler 16.indis class (outcome) alindi
etiketler=np_utils.to_categorical(etiketler, 7)
# Class isimleri 0,1,2,3,...6 iken 7 giris icin ayarlandi 
# 1= [1,0,0,0,0,0,0], 2=[0,1,0,0,0,0,0], ...

# Splitting data 
X_train, X_test, Y_train, Y_test = train_test_split(datalar, etiketler, test_size=0.2, random_state=0)
#X_train: Train Datalari
#X_test:  Test Dataları
#Y_train: Train Etiketleri
#Y_test:  Test Etiketleri   

In [ ]:
print(etiketler.shape)
print(etiketler[13600,:])
#print(datalar.shape)
#print(datalar)

(13611, 7)
[0. 0. 0. 0. 0. 0. 1.]


**MLP Yapisi Kurulumu**

In [ ]:
model=Sequential()
#8 feature girisli oldugu icin input_dim=16 secildi
model.add(Dense(80, input_dim=16, activation='tanh'))   # Hidden Layer 1 (90 noron)
#hidden layer 2 ye input_dim yazilmaz.
model.add(Dense(100, activation='sigmoid'))            # Hidden Layer 2  (130 noron)
model.add(Dense(140, activation='tanh'))            # Hidden Layer 2  (130 noron)
model.add(Dense(200, activation='sigmoid'))            # Hidden Layer 2  (130 noron)



#Cikis 7 adet oldugu icin oldugu icin Dense 7 output noron baglandi

model.add(Dense(7, activation='swish'))                #Output Layer
#activation=sigmoid, relu, linear, tanh, softmax,elu,

#Modeli Derleme
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
#loss=KLD, mae, mse, mean_squared_error, cosine_proximity, binary_crossentropy, poisson, categorical_crossentropy
#optimizer: adam, RMSprop, SGD

#batch_size: Egitim sirasinda verisetinden ornekler kacar kacar alinacak
model.fit(X_train, Y_train, epochs=500, batch_size=10, verbose=1)
#verbose=0 -> Eğitimi adim adim gostermez sadece son epoch getirir
basarim=model.evaluate(X_train, Y_train)

print("\n Train %s : %.2f%% " % (model.metrics_names[1], basarim[1]*100))


Epoch 1/500
1089/1089 [==============================] - 3s 2ms/step - loss: 0.0833 - accuracy: 0.6094
Epoch 2/500
1089/1089 [==============================] - 2s 2ms/step - loss: 0.0306 - accuracy: 0.8809
Epoch 3/500
1089/1089 [==============================] - 2s 2ms/step - loss: 0.0258 - accuracy: 0.8980
Epoch 4/500
1089/1089 [==============================] - 2s 2ms/step - loss: 0.0238 - accuracy: 0.9016
Epoch 5/500
1089/1089 [==============================] - 2s 2ms/step - loss: 0.0226 - accuracy: 0.9048
Epoch 6/500
1089/1089 [==============================] - 2s 2ms/step - loss: 0.0206 - accuracy: 0.9123
Epoch 7/500
1089/1089 [==============================] - 2s 2ms/step - loss: 0.0198 - accuracy: 0.9147
Epoch 8/500
1089/1089 [==============================] - 2s 2ms/step - loss: 0.0196 - accuracy: 0.9142
Epoch 9/500
1089/1089 [==============================] - 2s 2ms/step - loss: 0.0207 - accuracy: 0.9035
Epoch 10/500
1089/1089 [==============================] - 2s 2ms/step - l

In [ ]:
model.summary()
#model.get_config()
#from keras.utils.vis_utils import plot_model
#plot_model(model, to_file='model.png')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                425       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               2600      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 707       
Total params: 3,732
Trainable params: 3,732
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.save("98.03.h5")

**Confusion Matrix**

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

_, test_acc = model.evaluate(X_test, Y_test, verbose=0)
print('>Test Accuracy: %.2f%%' % (test_acc * 100.0))


#yPredicted = model.predict(X_test)           #Direk cikista alinan degerleri verir
#yPredicted = model.predict_proba(X_test)     #Olasiliksal olarak cikti verir
yPredicted = model.predict_classes(X_test)
Y_Test_actual=numpy.argmax(Y_test,axis=1)

print(classification_report(y_true=Y_Test_actual, y_pred=yPredicted))
print(confusion_matrix(y_true=Y_Test_actual, y_pred=yPredicted))


>Test Accuracy: 91.99%
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       376
           1       0.97      0.89      0.92       255
           2       1.00      1.00      1.00        92
           3       0.94      0.94      0.94       350
           4       0.92      0.97      0.95       390
           5       0.83      0.88      0.85       525
           6       0.94      0.90      0.92       735

    accuracy                           0.92      2723
   macro avg       0.94      0.93      0.93      2723
weighted avg       0.92      0.92      0.92      2723

[[356   1   0   0   0  18   1]
 [  2 226   0  15   2  10   0]
 [  0   0  92   0   0   0   0]
 [  1   6   0 330  10   3   0]
 [  0   0   0   5 379   5   1]
 [  7   1   0   1  14 461  41]
 [ 10   0   0   0   5  59 661]]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
